In [1]:
# System imports
import os
from dotenv import load_dotenv

# Python libraries
import requests

# Gradio imports
import gradio as gr

# OpenAI imports
from openai import OpenAI

/Users/winsonchow/Documents/GitHub/RAG-app/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/winsonchow/Documents/GitHub/RAG-app/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the environment variables
load_dotenv()

# Retrieve the API key from the environment variable
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [3]:
# Define API endpoints
list_posts_url = 'https://www.techinasia.com/wp-json/techinasia/2.0/posts'
search_posts_url = 'https://www.techinasia.com/wp-json/techinasia/2.0/articles?query='

In [4]:
# Function to fetch posts from API
def fetch_posts_from_api(page):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(list_posts_url, params={'page': page}, headers=headers)
        response.raise_for_status()
        print("Status Code:", response.status_code)
        # print("Response Headers:", response.headers)
        # print("Response Text:", response.text)

        post_data = response.json()

        return post_data['posts']
    
    except requests.exceptions.RequestException as e:
        print(f'Error fetching posts: {e}')
        return None
    except ValueError as e:
        print(f'Error parsing response: {e}')
        return None

In [5]:
# Function to search posts from API
def search_posts(query):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(search_posts_url, params={'query': query}, headers=headers)
        response.raise_for_status()
        print("Status Code:", response.status_code)
        # print("Response Headers:", response.headers)
        # print("Response Text:", response.text)

        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(f'Error fetching posts: {e}')
        return None
    except ValueError as e:
        print(f'Error parsing response: {e}')
        return None

In [6]:
# Fetch posts
page = 1
posts = fetch_posts_from_api(page=page)
print(f"Fetched {len(posts)} posts.")

# # Display the first post to verify the structure
# if posts:
#     print(posts[0])

Status Code: 200
Fetched 30 posts.


In [7]:
# # Search for posts with a keyword
# query = "What are the upcoming Events at Tech in Asia?"

# search_results = search_posts(query)

# if search_results and 'posts' in search_results and 'hits' in search_results['posts']:
#     search_posts = search_results['posts']['hits']
#     # context with title only
#     context = " ".join([post['title'] for post in search_posts[:10]])

#     # context with title and content
#     # context = "".join([post['title'] + post['content'] for post in search_posts[:3]])

#     print(f"Context: {context}")
# else:
#     context = 'No relevant posts found. Please provide more context.'

In [8]:
# Function to generate responses using OpenAI
def generate_response(search_results, query):
    try:
        if search_results and 'posts' in search_results and 'hits' in search_results['posts']:
            search_posts = search_results['posts']['hits']
            # context with title only
            # context = " ".join([post['title'] for post in search_posts[:2]])

            # context with title and content
            context = "".join([post['title'] + post['content'] for post in search_posts[:3]])
        else:
            context = 'No relevant posts found. Please provide more context.'

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Answer the question based on the question asked and context provided below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"},
                {"role": "user", "content": f"Question: {query}\nContext: {context}\nAnswer:"}
            ],
            max_tokens=100
        )
        return response.choices[0].message.content
    
    except Exception as e:
        print(f'Error generating response: {e}')
        return None

In [9]:
# # Test the response generation
# query = 'How is Grab performing in the market?'

# search_results = search_posts(query)
# generate_response(search_results, query)

In [10]:
# Build the Gradio interface
def rag_interface(query):
    search_results = search_posts(query)
    return generate_response(search_results, query)

iface = gr.Interface(fn=rag_interface, inputs="text", outputs="text", title='Tech in Asia RAG System', description='Ask a question about Tech in Asia and get an answer based on the context of the latest posts.')
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Status Code: 200
